In [37]:
using DataFrames, DataFramesMeta, LinearAlgebra, Distributions, Optim, Revise, Plots

In [38]:
string(@__DIR__) in LOAD_PATH || push!(LOAD_PATH, @__DIR__)
using BLPmodule

In [40]:
I = 1000;              # Number of consumers
J = 2;                  # Number of firms
K = 1;                  # Product characteristics
nT = 5;                 # Number of markets
# β = [1, -1];            # Preferences
β = -1.;            # Preferences
varζ = 3;               # Variance of the random taste
rangeB = [100, 1000];       # consumers per market (e.g. census block)
# varX = 2;               # Variance of X
varξ = 1;               # Variance of xi
δ0 = zeros(1, size(ζ, 2))
ζ = rand(Normal(0,1), 1, I); # Draw shocks (less)

In [41]:
dist_thresh = 0.5
df = BLPmodule.build_dist_data(J, β, nT, rangeB, varζ, varξ, dist_thresh)
df[:, Cols("t", "j", r"q", r"B", r"agg")]

,t,j,q,q0,agg_qj,agg_q0,B,agg_B,agg_s,agg_s0
,Int64,Int64,Float64,Float64,Float64,Float64,Int64,Int64,Float64,Float64
1,1,2,340.751,359.249,1138.74,1260.48,700,2722,0.418348,0.463073
2,2,2,47.3547,52.6453,1138.74,1260.48,100,2722,0.418348,0.463073
3,3,1,283.289,339.565,322.772,388.145,902,1042,0.309762,0.3725
4,3,2,279.145,339.565,1138.74,1260.48,902,2722,0.418348,0.463073
5,4,2,419.555,460.445,1138.74,1260.48,880,2722,0.418348,0.463073
6,5,1,39.4832,48.5796,322.772,388.145,140,1042,0.309762,0.3725
7,5,2,51.9371,48.5796,1138.74,1260.48,140,2722,0.418348,0.463073


In [ ]:
BLPmodule.compute_delta();

In [42]:
df_j = df[Not(findall(nonunique(df[!,[:j]]))), Cols("j", r"dist", r"agg")]

,j,dist,mean_dist,agg_qj,agg_q0,agg_B,agg_s,agg_s0
,Int64,Float64,Float64,Float64,Float64,Int64,Float64,Float64
1,2,0.153358,0.119416,1138.74,1260.48,2722,0.418348,0.463073
2,1,0.268196,0.332926,322.772,388.145,1042,0.309762,0.3725


In [33]:
using BLPmodule
deltas = BLPmodule.loop(df_j.agg_qj);

iterations:2


In [36]:
BLPmodule.simple_shares(deltas, df_j.mean_dist, ζ)

(3, 10000)


2-element Vector{Float64}:
 947.8096503540006
 219.48668005075152

In [35]:
println(BLPmodule.simple_shares(deltas))
df_j.agg_qj

[934.2249437267444, 210.84383709896815]


2-element Vector{Float64}:
 934.2249437267444
 210.84383709896807

In [26]:
deltas = BLPmodule.loop(df_j.agg_s);

iterations:2


In [27]:
println(BLPmodule.simple_shares(deltas))
df_j.agg_s

[0.42686377759135685, 0.42484988881288455]


2-element Vector{Float64}:
 0.42686377759135685
 0.42484988881288455

---

Stuff below are wrong (working with the original functions, trying to work with shares). 

In [51]:
aux_x = repeat([0.], J);
aux_t = repeat([1], J);

In [52]:
using BLPmodule
deltas = BLPmodule.compute_delta(df_j.agg_s, df_j.agg_s0, aux_x , ζ_, aux_t);
shares = BLPmodule.implied_shares(aux_x, ζ_, deltas, δ0)
println("computed shares:       ", shares)
println("true aggregate shares: ", df_j.agg_s)

final counter: 100001
deltas:[10.906520560496855, 10.724932478798397]
computed shares:       [0.5452672337486898, 0.4547227669949113]
true aggregate shares: [0.3657848442956161, 0.30504436398899093]


In [53]:
# using quantities
using BLPmodule
deltas = BLPmodule.compute_delta(df_j.agg_qj, df_j.agg_q0, aux_x, ζ_, aux_t);
shares = BLPmodule.implied_shares(aux_x, ζ_, deltas, δ0)
println("computed shares:       ", shares)
println("true aggregate shares: ", df_j.agg_s)

final counter: 100001
deltas:[11.106302651709791, 10.416817206024435]
computed shares:       [0.6658457937795371, 0.3341442069640639]
true aggregate shares: [0.3657848442956161, 0.30504436398899093]


In [224]:
# δ0 = zeros(1, size(ζ_, 2))
# for t in unique(T)
#     tind = df.t.==t
#     shares = BLPmodule.implied_shares(X_[tind,:], ζ_, deltas1, δ0)
#     println("computed shares: ", shares)
# end
# println(agg_shares)

In [225]:
# using BLPmodule
# Minimize GMM objective function
# varζ_ = optimize(x -> BLPmodule.GMM(q_j_, q0_, X_, Z_, ζ_, T, IV_, x[1])[1], [3.0], LBFGS()).minimizer[1];

In [226]:
# β_blp = BLPmodule.GMM(q_j_, q0_, X_, Z_, ζ_, T, IV_, varζ_)[2];

In [227]:
# print("Estimated BLP coefficients: $β_blp")

In [228]:
# # Compute logit estimate
# y = log.(df.s) - log.(df.s0)
# β_logit = inv(IV_' * X_) * (IV_' * y)
# print("Estimated logit coefficients: $β_logit")